In [1]:
⋅ = *

* (generic function with 362 methods)

# Lemma 1.2

$$
    \int_{a}^{b} f(x)\ dx = \int_{g^{-1}(a)}^{g^{-1}(b)} f(g(u))g'(u)\ du \tag{p. 5}
$$

but this is not what the lemma on p. $7$ is saying. The lemma is saying that, given limits of integration a(t) and b(t)

$$
    \frac{d}{dt} \left( \int_{a(t)}^{b(t)} f(x)\ dx \right) = f(b(t))b'(t) - f(a(t))a'(t)
$$

which can be proven using the Fundamental Theorem of Calculus.

# Payoff of Call and Put Options

$$
    C(T) = \max(S(T) - K, 0) =
    \begin{cases}
        S(T) - K & S(T) \gt K \\
        0        & S(T) \leq K
    \end{cases}
    \\~\\
    P(T) = \max(K - S(T), 0) =
    \begin{cases}
        0        & S(T) \geq K \\
        K - S(T) & S(T) \lt K
    \end{cases}
$$

# Put-Call Parity

The Put-Call Parity usually goes as follows:
$$
    P(t) + S(t) - C(t) = K\ e^{-r(T-t)} \tag{1}
$$

The **Law of One Price** can be used to prove this important theorem (cf. page 19). It is important to note that any violation of the Put-Call parity would present an **arbitrage opportunity**. It is worth reviewing the **Fundamental Theorem of Asset Pricing**.

# Forward and Futures Contracts

## Lemma 1.7 from Book

If the value $V(T)$ of a portfolio at time $T$ in the future is independent of the state of the market at time $T$, then

$$
    V(t) = V(T)\ e^{-r(T - t)} \tag{2}
$$

## Non-Dividend Paying Asset

$$
    F = S(0)\ e^{rT} \tag{3}
$$

## Dividend Paying Asset

$$
    F = S(0)\ e^{(r - q)T} \tag{4}
$$

The future contract has a similar structure as above, the only difference being that the underlying asset wold require delivery for the futures price. Forward contracts can be **settled in cash** without delivery of any physical asset at maturity.


## Other Take Home Results

$$
    F(t) = S(t) - K\ e^{-r(T - t)} \\
    \ \ \ \ \ \ \ \ \ \ \ \ \ F(t) = S(t)\ e^{-q(T-t)} - K\ e^{-r(T - t)}
$$

# Forward Contracts and the Put-Call Parity

$
    C(T) - P(T) = \max((S(T) - K), 0) - \max((K - S(T)), 0) \\~\\
    \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ =
    \begin{cases}
        0 - (K - S(T)) = S(T) - K & S(T) \leq K \\
        (S(T) - K) - 0 = S(T) - K & S(T) \geq K
    \end{cases} \\~\\
    \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ = S(T) - K \\
    \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ = F(T)
$

# Numerical Integration Methods (Riemann)

We start by defining $n$ to be the number of **partition intervals**. Then, for $I = \int_{a}^{b}f(x)\ dx$, we have $h = \frac{b - a}{n}\ \forall\ a_i = a + ih \in [a, b]$.

## Midpoint Rule

$$
    I^M_n = \sum^n_{i = 1}\int_{a_{i - 1}}^{a_i} c_i(x)\ dx = h\sum^n_{i = 1} f(x_i),\ \mathrm{where}\ x_i = \frac{a_{i - 1} + a_{i}}{2} \tag{5}
$$

In [18]:
function Midpoint(a, b, n, f_int)::Float64
    h = (b - a) / n
    I_midpoint = 0
    for i in 1:n
        I_midpoint += f_int(a + (i - 0.5) ⋅ h)
    end
    return h ⋅ I_midpoint
end

Midpoint (generic function with 1 method)

## Trapezoidal Rule

$$
    I^T_n = \sum^n_{i = 1}\int_{a_{i - 1}}^{a_i} l_i(x) dx  = \frac{h}{2} \left(f(a_0) + 2\sum_{i = 1}^{n - 1} f(a_i) + f(a_n) \right) \tag{6}
$$

In [19]:
function Trapezoidal(a, b, n, f_int)::Float64
    h = (b - a) / n
    I_trap = f_int(a) / 2 + f_int(b) / 2
    for i in 1:n
        I_trap += f_int(a + i ⋅ h)
    end
    return h * I_trap
end

Trapezoidal (generic function with 1 method)

## Simpson's Rule

$$
    I^S_n = \sum_{i = 1}^n \int_{a_{i - 1}}^{a_i} q_i(x) dx = \frac{h}{6}\left(f(a_0) + 2\sum_{i = 1}^{n - 1} f(a_i) + f(a_n) + 4\sum_{i = 1}^nf(x_i)\right) \tag{7}
$$

In [20]:
function Simpson(a, b, n, f_int)::Float64
    h = (b - a) / n
    I_simpson = f_int(a) / 6 + f_int(b) / 6
    for i in 1:(n - 1)
        I_simpson += f_int(a + i ⋅ h) / 3
    end
    for i in 1:n
        I_simpson += 2 ⋅ f_int(a + (i - 0.5) ⋅ h) / 3
    end
    return h ⋅ I_simpson
end

Simpson (generic function with 1 method)

## Numerical Approximation for Any Method

In [7]:
function I_numerical(a, b, f_int, method, tol)
    no_intervals = []
    n = 4; I_old = method(a, b, n, f_int)
    append!(no_intervals, n)
    n = 2n; I_new = method(a, b, n, f_int)
    append!(no_intervals, n)
    I_approx = [I_old]
    while(abs(I_new - I_old) > tol)
        I_old = I_new
        n = 2n
        append!(no_intervals, n)
        append!(I_approx, I_new)
        I_new = method(a, b, n, f_int)
    end
    append!(I_approx, I_new)
    return no_intervals, I_approx
end

I_numerical (generic function with 1 method)

## Relationship between Simpson's, Midpoint and Trapezoidal Rules

$$
    I^S_n = \frac{2}{3} I^M_n + \frac{1}{3} I^T_n \tag{8}
$$

## Example

In [21]:
using DataFrames

f(x) = exp(-x^2)
# numericals = Array{Float64, 2}(undef, 15, 4)
numericals = zeros(15, 4)
no_intervals, I_midpoint = I_numerical(0, 2, f, Midpoint, 1e-6)
no_intervals, I_trap = I_numerical(0, 2, f, Trapezoidal, 1e-6)
no_intervals, I_simpson = I_numerical(0, 2, f, Simpson, 1e-6)

no_intervals = [4 ⋅ 2^i for i in 0:max(size(I_midpoint)[1], size(I_trap)[1], size(I_simpson)[1])-1]

for i in 1:max(size(I_midpoint)[1], size(I_trap)[1], size(I_simpson)[1])
    numericals[i, 1] = no_intervals[i]
end

for i in 1:size(I_midpoint)[1]
    numericals[i, 2] = I_midpoint[i]
end

for i in 1:size(I_trap)[1]
    numericals[i, 3] = I_trap[i]
end

for i in 1:size(I_simpson)[1]
    numericals[i, 4] = I_simpson[i]
end

df = DataFrame(Intervals = no_intervals, Midpoint = numericals[:, 2], Trapezoidal = numericals[:, 3], Simpson = numericals[:, 4])

,Intervals,Midpoint,Trapezoidal,Simpson
,Int64,Float64,Float64,Float64
1,4,0.882789,0.889776,0.882066
2,8,0.882269,0.886283,0.88208
3,16,0.882129,0.884276,0.882081
4,32,0.882093,0.883202,0.0
5,64,0.882084,0.882648,0.0
6,128,0.882082,0.882366,0.0
7,256,0.882082,0.882224,0.0
8,512,0.0,0.882153,0.0
9,1024,0.0,0.882117,0.0


## Approximation Errors

$$
    O\left(\frac{1}{n^2}\right) = |I - I_{n}^M| \leq \frac{h^2}{24}(b - a)\ \max_{a \leq x \leq b}|f''(x)| = \frac{1}{n^2}\frac{(b - a)^3 M_2}{24}, \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \textrm{where}\ M_2 = \max_{a \leq x \leq b}|f''(x)| \lt \infty \tag{9}
$$

$$
    O\left(\frac{1}{n^2}\right) = |I - I_{n}^T| \leq \frac{h^2}{12}(b - a)\ \max_{a \leq x \leq b}|f''(x)| = \frac{1}{n^2}\frac{(b - a)^3 M_2}{12}, \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \textrm{where}\ M_2 = \max_{a \leq x \leq b}|f''(x)| \lt \infty \tag{10}
$$

$$
    O\left(\frac{1}{n^4}\right) = |I - I_{n}^S| \leq \frac{h^4}{2880}(b - a)\ \max_{a \leq x \leq b}|f^{(4)}(x)| = \frac{1}{n^4}\frac{(b - a)^5 M_4}{2880}, \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \textrm{where}\ M_4 = \max_{a \leq x \leq b}|f^{(4)}(x)| \lt \infty \tag{11}
$$

In [14]:
using QuadGK
approx_error = function(f, M, T, S)
    F = quadgk(f, 0, 2, rtol=1e-7)[1]
    return (abs(int(F - M)), abs(int(F - T)), abs(int(F - S)))
end

#1 (generic function with 1 method)

# Bond Pricing, Duration and Convexity

The following are results from the **Fundamental Theorem of Asset Pricing**. The assumption here is that asset distributions are lognormal. A Jarque-Bera Normality Test would prove worthy here for price $\Delta$'s.

## Interest rate curves

The values of $B(0)$ currency units and $B(t)$ currency units, are

$$
    B(t) = \exp{(t r(0, t)}\ B(0) \tag{12}
$$

$$
    \ \ \ B(0) = \exp{(-t r(0, t))}\ B(t) \tag{13}
$$

for times $t$ and $0$, respectively.

## Zero rates and instantaneous rates.

$$
    r(t) = \lim_{dt \rightarrow 0}\ \frac{1}{dt} \cdot \frac{B(t + dt) - B(t)}{B(t)} = \frac{B'(t)}{B(t)} \tag{14}
$$

$\therefore$ $B(t)$ satisfies the ordinary differential equation

$$
    \frac{B'(τ)}{B(τ)} = r(τ),\ \forall\ τ > 0 \tag{15}
$$

with $B(τ) = B(0)$ at time $τ = 0$. It follows that

$$
    \int^{t}_{0}{r(\tau)}\ d\tau = \int^{t}_{0} \frac{B'(\tau)}{B(\tau)}d\tau = \ln(B(\tau))|_{\tau = 0}^{\tau = t} = \ln(B(t)) - \ln(B(0)) = \ln \left(\frac{B(t)}{B(0)}\right) \tag{16}
$$

We then have that

$$
    B(t) = B(0)\ \exp\left(\int_{0}^{t}\ r(\tau)\ d\tau\right),\ \forall\ t \gt 0 \tag{17}
$$

$$
    B(0) = B(t)\ \exp\left(-\int_{0}^{t}\ r(\tau)\ d\tau\right),\ \forall\ t \gt 0 \tag{18}
$$

It follows that

$$
    r(0, t) = \frac{1}{t}\ \int_{0}^{t} r(τ)\ dτ \tag{19}
$$

If $r(t)$ is continuous, then $r(t)$ is $\textbf{uniquely}$ determined if the zero rate $r(0, t)$ is known. From $(19)$, we obtain that

$$
    \int_{0}^{t}\ r(\tau)\ d\tau = t\ r(0, t) \tag{20}
$$

From $(20)$, we find that (using the **Product Rule** and **Lemma 1.2** above)

$$
    r(t) = r(0, t) + t\ \frac{d}{dt}(r(0, t)) \tag{21}
$$

Finally,

$$
    B(t_2) = B(t_1)\ \exp\left(\int_{t_1}^{t_2}\ r(\tau)\ d\tau\right),\ \ \ \ \forall\ 0 \lt t_1 \lt t_2 \tag{22}
$$

$$
    B(t_1) = B(t_2)\ \exp\left(-\int_{t_1}^{t_2}\ r(\tau)\ d\tau\right),\ \forall\ 0 \ \lt t_1 \lt t_2 \tag{23}
$$

## Constant interest rates

If interest rates are constant and equal to $r$, the future value and present value formulas become

$$
    B(t) = e^{rt}\ B(0),\ \forall\ {t \gt 0} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{24}
$$

$$
    B(t_2) = e^{r(t_2 - t_1)}\ B(t_1),\ \forall\ {0 \lt t_1 \lt t_2} \ \tag{25}
$$

$$    
    B(0) = e^{-rt}\ B(t),\ \forall\ {t \gt 0} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{26}
$$

$$
    B(t_1) = e^{-r(t_2 - t_1)}\ B(t_2),\ \forall\ 0 \lt t_1 \lt t_2 \tag{27}
$$

**Review $\S$ 2.5.1 on pp. 52-53 for more on _constant interest rates_.**

## Forward rates

To determine the forward rate at $t_2$ of a coupon bond that has accumulated value up to time $t_1$, with a continuation deposit at $t_1$, we use the following

$$
    r(0; t_1, t_2) = \frac{t_2\ r(0, t_2) - t_1\ r(0, t_1)}{t_2 - t_1} \tag{28}
$$

In the text, they mention that, by the **Law of One Price**, this would yield the same value as depositing $B(0)$ at time $0$; i.e., for

$$
    V_1(t_2) = V_1(t_1)\ \exp(\ (t_2 - t_1)\ r(0; t_1, t_2)\ )
    = B(0)\ \exp(\ t_1\ r(0, t_1) + (t_2 - t_1)\ r(0; t_1, t_2)\ ). \tag{29}
$$

and

$$
    V_2(t_2) = B(0)\ \exp(t_2\ r(0, t_2)). \tag{30}
$$

both investment strategies are risk free and the cash amount invested at time $0$ is the same, equal to $B(0)$ for both strategies.

$$
    \therefore\ V_1(t_2) = V_2(t_2) \tag{31}
$$

**Review $\S 2.5.2$ (pp. 53-54) for more details on _Forward rates_.**

## Bond pricing. Yield of a bond.

If the **zero rate cuve** is known, the discount factor corresponding to time $t_i$ is $e^{-r(0, t_i)t_i}$.

$$
    ∴ B = \sum_{i = 1}^n c_ie^{-r(0, t_i)t_i} \tag{32}
$$

If the **instantaneous interest rate curve $r(t)$** is known, we can replace $e^{-r(0, t_i)t_i}$ with $\exp{\left(-\int_0^{t_i}r(τ)\ dτ\right)}$ in (32), which then gives us

$$
    ∴ B = \sum_{i = 1}^n c_i\ \exp{\left(-\int_0^{t_i}r(τ)\ dτ\right)} = F\ \exp{\left(-\int_0^Tr(\tau)d\tau\right)} \tag{33}
$$

where $F$ is the face value of the Bond (see $\S 2.9$ for a discussion on how this is the case and also on how $B = Fe^{-yT}$ if the yield of the bond is given, as well as modified duration $\left(D = -\frac{1}{Fe^{-yT}} = T\right)$ and convexity $\left(C = \frac{1}{Fe^{-yT}}(T^2 Fe^{-yT}) = T^2\right)$.

An **annual-based, periodic coupon bond** with face value $F$, coupon rate $C$ and maturity $T$ (usually measured in years) pays the holder of the bond a coupon payment equal to $\frac{C}{m} F$ (except at maturity), where $m$ is the coupon divisor. The final payment at maturity is equal to the face value of the bond plus one coupon payment, i.e., $F\left(1 + \frac{C}{m}\right)$. The total cashflows would then be

$$
    C_{total} = F \sum_{i = 1}^n \frac{C}{m} + F \tag{34}
$$

where $n$ is the number of payments. This is true for coupon payments and final payment

$$
    \left[F\overbrace{\left[\frac{C}{m}, \frac{C}{m},\ldots,\frac{C}{m}\right]}^\text{$n-1$ times},\ F + \frac{C}{m}F\right] \tag{35}
$$

and months

$$
    \left[T - m(n - 1), \ldots, T - \left\lfloor{\frac{T}{n}}\right\rfloor, T\right] \tag{36}
$$

## Zero rate algorithm

In [22]:
function bond_price_zero_rate(n, t_cash_flow, v_cash_flow, r_zero)
    disc = zeros(n)
    B = 0
    for i in 1:n
        disc[i] = exp(-t_cash_flow[i] ⋅ r_zero(t_cash_flow[i]))
        B = B + v_cash_flow[i] ⋅ disc[i]
    end
    return disc, B
end

bond_price_zero_rate (generic function with 1 method)

## Instantaneous rate algorithm

In [23]:
function bond_price_inst_rate(n, t_cash_flow, v_cash_flow, r_inst, tol)
    B = 0
    I_num = zeros(n)
    disc = zeros(n)
    for i in 1:n
        no_intervals, I_approx = I_numerical(0, t_cash_flow[i], r_inst, Simpson, tol[i])
        I_num[i] = I_approx[end]
        disc[i] = exp(-I_num[i])
        B = B + v_cash_flow[i] ⋅ disc[i]
    end
    return disc, B
end

bond_price_inst_rate (generic function with 1 method)

## Example

Consider a semiannual coupon bond with coupon rate $6\%$ and maturity $20$ months. Assume that the face value of the bond is $100$, and that the interest is compounded continuously.

**(I) Given zero rate**

$$
    r(0, t) = 0.0525 + \frac{\ln(1 + 2t)}{200}
$$

compute the price of the bond.

**Solution** We count backwards from $20$ months, i.e., $20$, $14$, $8$ and $2$, and get coupon payments for `[2, 8, 14, 20]`, i.e., `[3, 3, 3, 103]` for $\left[\frac{2}{12},\frac{8}{12},\frac{14}{12},\frac{20}{12}\right]$. In fact, the following function should take care of doing that for us.

In [24]:
function get_cash_flow(n, m, T, F, C, M)
    t_cash_flow = zeros(n)
    v_cash_flow = zeros(n)
    I = n; t = T
    while (I > 0)
        t_cash_flow[I] = t / M
        t = T - (M / m) ⋅ (n - I + 1) # (n - (I - 1))
        I -= 1
    end
    I = 1
    while (I < n)
        v_cash_flow[I] = F ⋅ (C / m)
        I += 1
    end
    v_cash_flow[n] = F ⋅ (1 + C / m)
    return t_cash_flow, v_cash_flow
end

get_cash_flow (generic function with 1 method)

Then we have

In [25]:
t_cash_flow, v_cash_flow = get_cash_flow(4, 2, 20, 100, 0.06, 12)

([0.166667, 0.666667, 1.16667, 1.66667], [3.0, 3.0, 3.0, 103.0])

and

In [27]:
f(t) = 0.0525 + log(1 + 2t) / 200
bond_price_zero_rate(4, t_cash_flow, v_cash_flow, f)

([0.991051, 0.962882, 0.934005, 0.905091], 101.88821588120076)

**(II) Given instantaneous rate**

$$
    r(t) = 0.0525 + \frac{1}{100(1 + e^{-t^2})}
$$

compute the price of the bond (see $\text{p.}\ 68$ for a discussion).

**Solution** Since the integral $\int r(τ) d\tau$ does not have a closed formula, it cannot be computed exactly. Therefore, we use Simpson's rule, and get the following discount factors and bond price for tolerance vector $ tol = [10^{-4}\ 10^{-4}\ 10^{-4}\ 10^{-6}]$.

In [28]:
tol = [10e-4, 10e-4, 10e-4, 10e-6]
g(t) = 0.0525 + 1 / (100(1 + exp(-t^2)))
bond_price_inst_rate(4, t_cash_flow, v_cash_flow, g, tol)

([0.990459, 0.962156, 0.933954, 0.905775], 101.9545638832744)

**(III) Given instantaneous rate**

$$
    r(t) = 0.0525 + \frac{\ln(1 + 2t)}{200} + \frac{t}{100(1 + 2t)}
$$

compute the price of the bond.

**Solution** Using the instantaneous bond pricing algorithm `bond_price_inst_rate` and Simpson's rule, we get the following bond prices (and respective discount factors) for two different tolerance vectors $tol_1 = [10^{-4} 10^{-4} 10^{-4} 10^{-6}]$ and $tol_2 = [10^{-4} 10^{-4} 10^{-4} 10^{-4}]$. 

In [33]:
tol_1 = [10e-4, 10e-4, 10e-4, 10e-6]; tol_2 = [10e-4, 10e-4, 10e-4, 10e-4]
h(t) = 0.0525 + log(1 + 2t) / 200 + t / (100(1 + 2t))
b1 = bond_price_inst_rate(4, t_cash_flow, v_cash_flow, h, tol_1);
b2 = bond_price_inst_rate(4, t_cash_flow, v_cash_flow, h, tol_2);
[BigFloat(b1[2]), BigFloat(b2[2]), BigFloat.(b1[1]), BigFloat.(b2[1])]

4-element Array{Any,1}:
 101.8882338846292867629017564468085765838623046875 
 101.8882338846292867629017564468085765838623046875 
    BigFloat[0.991051, 0.962882, 0.934005, 0.905091]
    BigFloat[0.991051, 0.962882, 0.934005, 0.905091]

I used `BigFloat`, since I wanted to have more accurate approximations of `b1` and `b2`, but I am still disatisfied with the outcome. Are not these two bond prices supposed to be different, given the two tolerance vectors?

The **yield of a bond** is the internal rate of return of the bond. After the accumulation of $n$ discounted future cash flows, we have the bond price $B$ as follows:

$$
    B = \sum_{i = 1}^n c_ie^{-yt_i} \tag{37}
$$

The price of the bond $B(y)$ is a decreasing function of the yield $y$:

$$
    \frac{\partial B}{\partial y} = B'(y) = -\sum_{i = 1}^{n}t_ic_ie^{-yt_i} \lt 0 \tag{38}
$$

In the text, one is invited to visit $\S 5.3$ of the book for details on computing bond yields, which goes as follows:

$$
    f(y) = 0, \ \ \ \ \ \ \ \ \textrm{where}\ f(y) = \sum_{i = 1}^nc_ie^{yt_i} - B \tag{39}
$$

where $y$ is a solution (may or may not be unique) to the nonlinear equation $f(y) = 0$. Review $\S 2.6$ (pp. 56-57) for more details on **yield** and something known as **par yield**.

## Bond duration and convexity

For a more elaborate discussion on bond duration and convexity, refer to $\S 2.7\ (\text{pp.}\ 57-59)$.

The **modified duration** of a bond is defined as follows:

$$
    D = -\frac{1}{B}\frac{\partial B}{\partial y} \tag{40}
$$

Using equation $(40)$, we can work the modified duration to be

$$
    D = \frac{1}{B}\sum_{i = 1}^nt_ic_ie^{-yt_i} \tag{41}
$$

The Macaulay duration of a bond is defined to be the weighted average of the cash flow times, with weights equal to the value of the corresponding cash flow discounted with respect to the ield of the bond; i.e.,

$$
    D_{Mac} = \frac{1}{B}\sum_{i = 1}^nt_ic_iDisc(t_i, y) \tag{42}
$$

For continuously compounded discounted cash flows, the Maclaury duration converges to the modified duration; i.e.,

$$
    D_{Mac} = \frac{1}{B}\sum_{i = 1}^nt_ic_ie^{-yt_i} \rightarrow D \approx -\frac{1}{B}\frac{\Delta B}{\Delta y} \tag{43}
$$

Terms could be rearranged in (22) to yield $\frac{\Delta B}{B} \approx -\Delta y\ D$. More discussion on p. 58 in text.

The **convexity** $C$ of a bond with price $B$ and yield $y$ is

$$
    C = \frac{1}{B}\ \frac{\partial^2 B}{\partial y^2} = \frac{1}{B}\sum_{i = 1}^{n}t_i^2c_ie^{-yt_i} \tag{44}
$$

Using Taylor expansion (see $\S 6.6$ for the proof), it can be shown that the following approximation for the return of a bond is more accurate.

$$
    \frac{\Delta B}{B} \approx -D\Delta y + \frac{1}{2}C(\Delta y)^2 \tag{45}
$$

**Review $\text{pp.}\ 59-64$ and $\text{pp.}\ 64-65$ for discussions on _discretely compounded interest_ and _zero coupon_ bonds respectively.**

In [55]:
function bond_PrDuCv(T, n, t_cash_flow, v_cash_flow, y)
    B = 0; D = 0; C = 0
    disc = zeros(n)
    for i in 1:n
        disc[i] = exp(-t_cash_flow[i]y)
        B = B + v_cash_flow[i]disc[i]
        D = D + t_cash_flow[i]v_cash_flow[i]disc[i]
        C = C + t_cash_flow[i]^2 * v_cash_flow[i] * disc[i]
    end
    D = D / B; C = C / B
    return B, D, C
end

bond_PrDuCv (generic function with 1 method)

## Example

This one is similar to the above example, except that instead of an interest rate we are given yield $6.50\%$ and we are asked to compute the bond price, duration and convexity.

**Solution** Using the code in `bond_PrDuCv` we use the same input as in the previous example and obtain

In [56]:
bond_PrDuCv(20, 4, t_cash_flow, v_cash_flow, 0.065)

(101.04619291323598, 1.5804215018374828, 2.5916858802726863)

# Black-Scholes Formulas

**Note: the Black-Scholes Formulas takes an heavy-weight assumption that assets have a log-normal distribution of prices.**

Using the Black-Scholes formulas below, we can derive the following `P` and `C` positions for European-style options, with volatility $\sigma$, paying dividends continuously at a rate $q$, with constant risk-free rate $r$, spot price $S$ (assumed to be real), strike $K$ (instantaneous wrt the position taken) and maturity $T$:

$$
    C(S, t) = Se^{-q(T\ -\ t)}N(d_1)\ -\ Ke^{-r(T - t)}N(d_2); \tag{46}
$$

$$
    \ \ \ \ P(S, t) = Ke^{-r(T\ -\ t)}N(-d_2)\ -\ Se^{-q(T - t)}N(-d_1) \tag{47}
$$

where the drift terms, $d_1$ and $d_2$ are defined as:

$$
    \ \ \ \ \ \ d_{1} = \frac{\ln\left(\frac{S}{K}\right)\ +\ \left(r\ -\ q\ +\ \frac{\sigma^2}{2}\right)(T\ -\ t)}{\sigma \sqrt{T\ -\ t}} \tag{48}
$$

$$
    \ \ \ \ \ \ d_{2} = d_1 - \sigma \sqrt{T - t}\ =\ \frac{\ln\left(\frac{S}{K}\right)\ +\ \left(r\ -\ q\ -\ \frac{\sigma^2}{2}\right)(T\ -\ t)}{\sigma \sqrt{T\ -\ t}} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{49}
$$

with cummulative normal densities defined as

$$
    N'(d_1) = \frac{1}{\sqrt{2\pi}} e^{-\frac{d_1^2}{2}} \tag{50}
$$

$$
    N'(d_2) = \frac{1}{\sqrt{2\pi}} e^{-\frac{d_2^2}{2}} \tag{51}
$$

## Black-Scholes Implementation

In [12]:
include("./aux.jl");

In [37]:
function d1(t, S, K, T, σ, r, q)
    return (log(S / K) + (r - q + σ^2 / 2) ⋅ (T - t)) / (σ ⋅ sqrt(T - t))
end

d1 (generic function with 1 method)

In [39]:
function d2(t, S, K, T, σ, r, q)
    return d1(t, S, K, T, σ, r, q) - σ ⋅ sqrt(T - t)
end

d2 (generic function with 1 method)

In [34]:
function cum_dist_normal(t)
    z = abs(t)
    y = 1 / (1 + 0.2316419z)
    a1 = 0.319381530; a2 = -0.356563782; a3 = 1.781477937; a4 = -1.821255978; a5 = 1.330274429
    
    m = 1 - exp(-t^2 / 2) * (a1 * y + a2 * y^2 + a3 * y^3 + a4 * y^4 + a5 * y^5) / √(2 * π)
    if t > 0
        nn = m
    else
        nn = 1 - m
    end
    return nn
end

cum_dist_normal (generic function with 1 method)

In [35]:
function black_scholes_approx(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q); d_2 = d2(t, S, K, T, σ, r, q)
    C = S * exp(-q * (T - t)) * cum_dist_normal(d_1) - K * exp(-r * (T - t)) * cum_dist_normal(d_2)
    P = K * exp(-r * (T - t)) * cum_dist_normal(-d_2) - S * exp(-q * (T - t)) * cum_dist_normal(-d_1)
    return [d_1, d_2], [C, P]
end

black_scholes_approx (generic function with 1 method)

## Example

In [9]:
t = 0; S = 42; K = 40; T = 0.5; σ = 0.3; q = 0.03; r = 0.05
d1d2, CP = black_scholes_approx(t, S, K, T, σ, r, q)

([0.383206, 0.171073], [4.70533, 2.34302])

# The Greeks of European Call and Put Options

$$
    \Delta(V) = \frac{\partial V}{\partial S} \ \ \tag{52}
$$

$$
    \Gamma(V) = \frac{\partial^{2} V}{\partial S^2} \tag{53}
$$

$$
    \Theta(V) = \frac{\partial V}{\partial t} \ \ \tag{54}
$$

$$
    \rho(V) = \frac{\partial V}{\partial r} \ \tag{55}
$$

$$
    \textrm{vega}(V) = \frac{\partial V}{\partial \sigma} \ \ \ \ \ \ \tag{56}
$$

$$
    \textrm{volga}(V) = \frac{\partial(\textrm{vega(V)})}{\partial \sigma} = \frac{\partial^2 V}{\partial \sigma^2} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{57}
$$

$$
    \textrm{vanna}(V) = \frac{\partial(\textrm{vega(V)})}{\partial S} = \frac{\partial^2 V}{\partial S \partial \sigma} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{58}
$$

## Magic of the Greeks

Given

$$
    d_1 = \frac{\ln\left(\frac{S}{K}\right) + (r - q)(T - t)}{σ\sqrt{T - t}} + \frac{σ\sqrt{T - t}}{2};\ d_2 = d_1 - \sqrt{T - t} \tag{59}
$$

we have that

$$
    Se^{q(T - t)}N'(d_1) = Ke^{-r(T - t)}N'(d_2) \tag{cf. Lemma 3.15 on p. 94; 60}
$$

where $N'(d_1) = \frac{1}{\sqrt{2π}}e^{-\frac{d_1^2}{2}}$ and $N'(d_2) = \frac{1}{\sqrt{2π}}e^{-\frac{d_2^2}{2}}$.

## Greeks for Call and Put Options

Again, one would use the Black-Scholes formulas to prove the following.

$$
    \Delta(C) = e^{-q(T\ -\ t)}N(d_1) \ \ \ \ \ \tag{61}
$$

In [15]:
function Delta_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q)
    return exp(-q(T - t)) * cum_dist_normal(d_1)
end

Delta_C (generic function with 2 methods)

$$
    \Delta(P) = -e^{-q(T\ -\ t)}N(-d_1) \tag{62}
$$

In [16]:
function Delta_P(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q)
    return -exp(-q(T - t)) * cum_dist_normal(-d_1)
end

Delta_P (generic function with 1 method)

$$
    \Gamma(C) = \frac{e^{-q(T\ -\ t)}}{S\sigma\sqrt{T\ -\ t}}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}} \tag{63}
$$

In [17]:
function Gamma_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q)
    return exp(-q(T - t)) / (S * σ * √(T - t)) * (1 / √(2π)) * exp(-d_1^2) / 2
end

Gamma_C (generic function with 1 method)

$$
    \Gamma(P) = \Gamma(C) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{64}
$$

In [18]:
function Gamma_P(t, S, K, T, σ, r, q)
    return Gamma_C(t, S, K, T, σ, r, q)
end

Gamma_P (generic function with 1 method)

$$
\Theta(C) = -\frac{S\sigma e^{-q(T\ -\ t)}}{2\sqrt{2\pi(T\ -\ t)}}\ +\ qSe^{-q(T\ -\ t)}N(d_1)\ -\ rKe^{-r(T\ -\ t)}N(d_2) \ \ \ \ \tag{65}
$$

In [23]:
function Theta_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q); d_2 = d2(t, S, K, T, σ, r, q)
    return -(S * σ * exp(-q * (T - t))) / (2 * √(2 * π * (T - t))) + q * S * exp(-q * (T - t)) * cum_dist_normal(d_1) - r * K * exp(-r * (T - t)) * cum_dist_normal(d_2)
end

Theta_C (generic function with 1 method)

$$
    \Theta(P) = -\frac{S\sigma e^{-q(T\ -\ t)}}{2\sqrt{2\pi(T\ -\ t)}}\ -\ qSe^{-q(T\ -\ t)} N(-d_1)\ +\ rKe^{-r(T\ -\ t)} N(-d_2) \tag{66}
$$

In [24]:
function Theta_P(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q); d_2 = d2(t, S, K, T, σ, r, q)
    return -(S * σ * exp(-q * (T - t))) / (2 * √(2 * π * (T - t))) - q * S * exp(-q * (T - t)) * cum_dist_normal(-d_1) + r * K * exp(-r * (T - t)) * cum_dist_normal(-d_2)
end

Theta_P (generic function with 1 method)

$$
    \rho(C) = K(T\ -\ t)e^{-r(T\ -\ t)}N(d_2) \ \ \ \ \ \ \tag{67}
$$

In [25]:
function Rho_C(t, S, K, T, σ, r, q)
    d_2 = d2(t, S, K, T, σ, r, q)
    return K * (T - t) * exp(-r * (T - t)) * cum_dist_normal(d_2)
end

Rho_C (generic function with 1 method)

$$
    \rho(P) = -K(T\ -\ t)e^{-r(T\ -\ t)}N(-d_2) \tag{68}
$$

In [26]:
function Rho_P(t, S, K, T, σ, r, q)
    d_2 = d2(t, S, K, T, σ, r, q)
    return -1 * K * (T - t) * exp(-r * (T - t)) * cum_dist_normal(-d_2)
end

Rho_P (generic function with 1 method)

$$
    \textrm{vega}(C) = Se^{-q(T\ -\ t)}\sqrt{T\ -\ t}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}} \tag{69}
$$

In [27]:
function Vega_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q)
    return S * exp(-q(T - t)) * √(T - t) * (1 / √(2π)) * exp(-d_1^2 / 2)
end

Vega_C (generic function with 1 method)

$$
    \textrm{vega}(P) = \textrm{vega}(C) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{70}
$$

In [29]:
function Vega_P(t, S, K, T, σ, r, q)
    return Vega_C(t, S, K, T, σ, r, q)
end

Vega_P (generic function with 1 method)

$$
    \textrm{volga}(C) = Se^{-q(T\ -\ t)}\sqrt{T\ -\ t}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}\frac{d_1 d_2}{\sigma} \tag{71}
$$

In [31]:
function Volga_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q); d_2 = d2(t, S, K, T, σ, r, q)
    return S * exp(-q * (T - t)) * √(T - t) * (1 / √(2π)) * exp(-d_1^2 / 2) * (d_1 * d_2 / σ)
end

Volga_C (generic function with 1 method)

$$
    \textrm{volga}(P) = \textrm{volga}(C) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{72}
$$

In [32]:
function Volga_P(t, S, K, T, σ, r, q)
    return Volga_C(t, S, K, T, σ, r, q)
end

Volga_P (generic function with 1 method)

$$
    \textrm{vanna}(C) = -e^{-q(T\ -\ t)}\frac{1}{\sqrt{2\pi}}e^{-\frac{{d_1}^2}{2}} \frac{d_2}{\sigma} \tag{73}
$$

In [33]:
function Vanna_C(t, S, K, T, σ, r, q)
    d_1 = d1(t, S, K, T, σ, r, q); d_2 = d2(t, S, K, T, σ, r, q)
    return -exp(-q(T - t)) * (1 / √(2π)) * exp(-d_1^2 / 2) * (d_2 / σ)
end

Vanna_C (generic function with 1 method)

$$
    \textrm{vanna}(P) = \textrm{vanna}(C) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \tag{74}
$$

In [34]:
function Vanna_P(t, S, K, T, σ, r, q)
    return Vanna_C(t, S, K, T, σ, r, q)
end

Vanna_P (generic function with 1 method)

## Example

(I) Using the Black-Scholes formula, we want to price a six month ($6\ \text{mo}$) European call option with strike 40, on an underlying asset with spot price $42$ and volatility $30\%$, which pays dividends **continuously**, with dividend rate $3\%$. Assume that interest rates are constant at $5\%$. (Also compute Greeks).

(II) Price a six month ($6\ \text{mo}$) European put option with strike 40 on the same asset, using the Black-Scholes formula. Check whether the Put-Call parity is satisfied (and check the Greeks).

In [40]:
# cum_dist_normal(t)
t = 0; S = 42; K = 40; T = 0.5; σ = 0.30; r = 0.05; q = 0.03
d1d2, CP = black_scholes_approx(t, S, K, T, σ, r, q)

([0.383206, 0.171073], [4.70533, 2.34302])